# =============================================================
# MILESTONE 2: Advanced Data Analysis and Feature Engineering
# =============================================================

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy import stats
from scipy.stats import chi2_contingency

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [17]:
# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [18]:
url = "../data/interim/initial_cleaned_data.csv"
df = pd.read_csv(url)

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


Dataset loaded: 3333 rows, 67 columns


# ========================
# 1. Statistical Tests
# ========================

In [19]:
# T-test for numerical features
print("\n--- T-test Results for Numerical Features ---")
numerical_cols = df.select_dtypes(include=[np.number]).columns.drop(['Churn', 'High_Customer_Service']).tolist()
t_test_results = []
for col in numerical_cols:
    group_churn = df[df['Churn'] == True][col]
    group_no_churn = df[df['Churn'] == False][col]
    t_stat, p_value = stats.ttest_ind(group_no_churn, group_churn, equal_var=False)
    significance = 'Significant' if p_value < 0.2 else 'Not Significant'  # Relaxed threshold
    print(f"{col:<25}: t-stat={t_stat:.3f}, p-value={p_value:.5f} -> {significance}")
    t_test_results.append({'Feature': col, 't-stat': t_stat, 'p-value': p_value, 'Significance': significance})


--- T-test Results for Numerical Features ---
Account length           : t-stat=-0.948, p-value=0.34333 -> Not Significant
Number vmail messages    : t-stat=5.821, p-value=0.00000 -> Significant
Total day minutes        : t-stat=-9.697, p-value=0.00000 -> Significant
Total day calls          : t-stat=-1.087, p-value=0.27731 -> Not Significant
Total eve minutes        : t-stat=-5.234, p-value=0.00000 -> Significant
Total eve calls          : t-stat=-0.472, p-value=0.63733 -> Not Significant
Total night minutes      : t-stat=-2.205, p-value=0.02780 -> Significant
Total night calls        : t-stat=-0.346, p-value=0.72950 -> Not Significant
Total intl minutes       : t-stat=-3.793, p-value=0.00016 -> Significant
Total intl calls         : t-stat=3.402, p-value=0.00071 -> Significant
Customer service calls   : t-stat=-7.589, p-value=0.00000 -> Significant


In [20]:
# Chi-squared tests for categorical features
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print("\n--- Chi-squared Test Results ---")
chi2_results = []
for col in categorical_cols:
    contingency_table = pd.crosstab(df[col], df['Churn'])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    significance = 'Significant' if p_value < 0.2 else 'Not Significant'  # Relaxed threshold
    chi2_results.append({'Feature': col, 'Chi2': chi2, 'p-value': p_value, 'Significance': significance})

chi2_df = pd.DataFrame(chi2_results).sort_values(by='p-value')
print(chi2_df)


--- Chi-squared Test Results ---
                   Feature        Chi2       p-value     Significance
52  International plan_Yes  222.565757  2.493108e-50      Significant
53     Voice mail plan_Yes   34.131660  5.150640e-09      Significant
30                State_NJ    7.082219  7.785253e-03      Significant
42                State_TX    5.720015  1.677259e-02      Significant
19                State_MD    4.757345  2.917352e-02      Significant
44                State_VA    3.435014  6.382803e-02      Significant
39                State_SC    3.162475  7.534888e-02      Significant
3                 State_CA    3.061232  8.018087e-02      Significant
2                 State_AZ    2.930757  8.690672e-02      Significant
21                State_MI    2.737304  9.803006e-02      Significant
10                State_HI    2.704046  1.000940e-01      Significant
24                State_MS    2.108423  1.464902e-01      Significant
32                State_NV    1.932201  1.645178e-01    

# ========================
# 2. Feature Engineering 
# ========================

In [21]:
# Create new features
# Customer tenure (normalize Account length)
df['Tenure'] = df['Account length'] / df['Account length'].max()

# Usage patterns: Total minutes and calls across all periods
df['Total_minutes'] = df['Total day minutes'] + df['Total eve minutes'] + df['Total night minutes'] + df['Total intl minutes']
df['Total_calls'] = df['Total day calls'] + df['Total eve calls'] + df['Total night calls'] + df['Total intl calls']

# Usage ratios
df['Day_minutes_per_call'] = df['Total day minutes'] / (df['Total day calls'] + 1)  # Add 1 to avoid division by zero
df['Eve_minutes_per_call'] = df['Total eve minutes'] / (df['Total eve calls'] + 1)
df['Night_minutes_per_call'] = df['Total night minutes'] / (df['Total night calls'] + 1)

# Interaction term: Customer service calls * International plan
df['Intl_plan_service_interaction'] = df['Customer service calls'] * df['International plan_Yes']

In [22]:
# Update numerical columns with new features
numerical_cols += ['Tenure', 'Total_minutes', 'Total_calls', 'Day_minutes_per_call', 
                   'Eve_minutes_per_call', 'Night_minutes_per_call', 'Intl_plan_service_interaction']

# ========================
# 3. Feature Selection
# ========================

In [23]:
# Select significant features (p < 0.2 + domain knowledge)
significant_features = [
    col for col in numerical_cols if col not in ['Account length'] or t_test_results[numerical_cols.index(col)]['p-value'] < 0.2
] + [
    col for col in categorical_cols if chi2_df[chi2_df['Feature'] == col]['p-value'].iloc[0] < 0.2
] + ['High_Customer_Service']
domain_features = ['Account length', 'Total day calls', 'Total eve calls', 'Total night calls']
significant_features = list(set(significant_features + domain_features))
print(f"\nSignificant features (p < 0.2 + domain): {len(significant_features)} features - {significant_features}")



Significant features (p < 0.2 + domain): 34 features - ['State_CA', 'Total intl calls', 'Intl_plan_service_interaction', 'State_MI', 'Night_minutes_per_call', 'Total day minutes', 'State_MS', 'State_AZ', 'State_TX', 'State_SC', 'High_Customer_Service', 'Total day calls', 'State_HI', 'Account length', 'Total night minutes', 'Total eve minutes', 'State_MD', 'Number vmail messages', 'State_WA', 'International plan_Yes', 'Voice mail plan_Yes', 'Total eve calls', 'Total_calls', 'Customer service calls', 'Tenure', 'State_VA', 'Eve_minutes_per_call', 'State_NJ', 'State_WV', 'Total night calls', 'Total_minutes', 'State_NV', 'Day_minutes_per_call', 'Total intl minutes']


In [24]:
# Remove highly correlated features (threshold 0.95)
corr_matrix = df[significant_features].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f"\nDropping highly correlated features: {to_drop}")
df = df.drop(to_drop, axis=1)
significant_features = [f for f in significant_features if f not in to_drop]


Dropping highly correlated features: ['Voice mail plan_Yes', 'Tenure']


In [25]:
# Scale numerical features
scaler = StandardScaler()
numerical_cols = [col for col in significant_features if col in df.select_dtypes(include=[np.number]).columns]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [26]:
# RFE with Random Forest
X = df[significant_features]
y = df['Churn']
model = RandomForestClassifier(random_state=42)
rfe = RFE(model, n_features_to_select=15)  # Select top 15 features
rfe.fit(X, y)
selected_features = X.columns[rfe.support_].tolist()
print(f"\nSelected features by RFE (Random Forest): {selected_features}")


Selected features by RFE (Random Forest): ['Total intl calls', 'Night_minutes_per_call', 'Total day minutes', 'High_Customer_Service', 'Total day calls', 'Total night minutes', 'Total eve minutes', 'Number vmail messages', 'International plan_Yes', 'Total_calls', 'Customer service calls', 'Eve_minutes_per_call', 'Total_minutes', 'Day_minutes_per_call', 'Total intl minutes']


In [27]:
# Feature importance for validation
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X, y)
importances = pd.Series(model_rf.feature_importances_, index=X.columns)
print("\nFeature Importances (Random Forest):")
print(importances.sort_values(ascending=False))


Feature Importances (Random Forest):
Total day minutes                0.148189
Total_minutes                    0.134087
Day_minutes_per_call             0.062531
Customer service calls           0.061954
Total intl minutes               0.055818
International plan_Yes           0.050120
High_Customer_Service            0.049953
Total eve minutes                0.046798
Total intl calls                 0.042666
Total night minutes              0.039947
Eve_minutes_per_call             0.038143
Number vmail messages            0.035074
Total day calls                  0.032366
Night_minutes_per_call           0.032313
Total_calls                      0.032033
Account length                   0.030437
Total eve calls                  0.029224
Total night calls                0.028663
Intl_plan_service_interaction    0.024027
State_TX                         0.004182
State_NJ                         0.003355
State_MS                         0.002935
State_MD                         0.002

In [28]:
# Save processed dataset
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)
df_selected = df[selected_features + ['Churn']]
df_selected.to_csv(os.path.join(output_dir, "processed_data.csv"), index=False)
print(f"\nProcessed dataset saved: {os.path.join(output_dir, 'processed_data.csv')}")


Processed dataset saved: ../data/processed\processed_data.csv


In [29]:
df_selected.shape

(3333, 16)

# ========================
# 3. Data Visualization
# ========================

In [13]:
import os
import plotly.express as px
import plotly.graph_objects as go

# Ensure the output directory exists
output_dir = "../visualizations/interactive"
os.makedirs(output_dir, exist_ok=True)

# 1. Churn Rate Pie Chart
fig_pie = px.pie(df, names='Churn', title='Churn Rate Distribution', color='Churn',
                 color_discrete_map={False: 'lightblue', True: 'salmon'})
fig_pie.write_html(os.path.join(output_dir, "churn_rate_pie.html"))
fig_pie.show()

# 2. Churn by State (Top 10 states with most churn)
state_cols = [col for col in df.columns if col.startswith('State_')]
df_states = df[state_cols + ['Churn']].copy()
df_states['State'] = df_states[state_cols].idxmax(axis=1).str.replace('State_', '')
churn_by_state = df_states[df_states['Churn'] == True]['State'].value_counts().nlargest(10)
fig_bar = px.bar(churn_by_state, x=churn_by_state.index, y=churn_by_state.values,
                 labels={'x': 'State', 'y': 'Churned Customers'},
                 title='Top 10 States by Churn Count', 
                 color=churn_by_state.values, 
                 color_continuous_scale='Reds')
fig_bar.write_html(os.path.join(output_dir, "top10_states_churn.html"))
fig_bar.show()

# # 3. Distribution of Average Daily Usage by Churn
# fig_hist = px.histogram(df, x='average daily usage', color='Churn', nbins=40, barmode='overlay',
#                         title='Distribution of Average Daily Usage by Churn',
#                         color_discrete_map={False: 'lightblue', True: 'salmon'})
# fig_hist.write_html(os.path.join(output_dir, "avg_daily_usage_by_churn.html"))
# fig_hist.show()

# # 4. Scatter Plot: Total Minutes vs. Total Calls colored by Churn
# fig_scatter = px.scatter(df, x='Total_Minutes', y='Total_Calls', color='Churn',
#                          title='Total Minutes vs. Total Calls by Churn',
#                          color_discrete_map={False: 'lightblue', True: 'salmon'},
#                          hover_data=['Customer tenure'])
# fig_scatter.write_html(os.path.join(output_dir, "minutes_vs_calls_by_churn.html"))
# fig_scatter.show()

# # 5. Box Plot: Customer Tenure by Churn
# fig_box = px.box(df, x='Churn', y='Customer tenure', color='Churn',
#                  title='Customer Tenure by Churn',
#                  color_discrete_map={False: 'lightblue', True: 'salmon'})
# fig_box.write_html(os.path.join(output_dir, "tenure_by_churn.html"))
# fig_box.show()

d:\Users\LENOVO\anaconda3\Lib\site-packages\kaleido\_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




# -------------------------------------
# 3.2.dashboard
# -------------------------------------

In [14]:
# Color scheme
colors = {False: "royalblue", True: "crimson"}

# Pie chart: Churn distribution
fig_pie = px.pie(
    df_selected, names="Churn", color="Churn",
    color_discrete_map=colors, hole=0.4
)
fig_pie.update_traces(textinfo="percent+label", pull=[0, 0.05])

# Bar chart: Significant categorical features
sig_cat_cols = [col for col in selected_features if col in df.select_dtypes(exclude=[np.number]).columns]
if sig_cat_cols:
    churn_by_cat = df_selected[sig_cat_cols + ['Churn']].melt(id_vars='Churn', var_name='Feature', value_name='Value')
    churn_by_cat = churn_by_cat[churn_by_cat['Value'] == 1].groupby(['Feature', 'Churn']).size().reset_index(name='Count')
    fig_bar = px.bar(
        churn_by_cat, x='Feature', y='Count', color='Churn',
        color_discrete_map=colors, barmode='group'
    )
    fig_bar.update_layout(xaxis_tickangle=-45)
else:
    fig_bar = px.bar(title="No significant categorical features selected")

# Histogram: Top numerical feature
numerical_cols = [col for col in selected_features if col in df.select_dtypes(include=[np.number]).columns]
if numerical_cols:
    top_num_feature = importances[numerical_cols].idxmax() if numerical_cols else numerical_cols[0]
    fig_hist = px.histogram(
        df_selected, x=top_num_feature, color="Churn", nbins=40, barmode="overlay",
        color_discrete_map=colors
    )
    fig_hist.update_traces(opacity=0.7)
else:
    fig_hist = px.histogram(title="No numerical features selected")

# Combine into dashboard
dashboard = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "domain"}, {"type": "xy"}], [{"type": "xy"}, {"type": "xy"}]],
    horizontal_spacing=0.12, vertical_spacing=0.15
)

for trace in fig_pie.data:
    dashboard.add_trace(trace, row=1, col=1)
for trace in fig_bar.data:
    dashboard.add_trace(trace, row=1, col=2)
for trace in fig_hist.data:
    dashboard.add_trace(trace, row=2, col=1)

# Add subplot titles
titles = ["Churn Distribution", "Significant Categorical Features", "Top Numerical Feature Distribution"]
positions = [(0.18, 1.05), (0.82, 1.05), (0.18, 0.70)]
for title, (x, y) in zip(titles, positions):
    dashboard.add_annotation(
        text=title, x=x, y=y, xref="paper", yref="paper",
        showarrow=False, font=dict(size=15, family="Arial", color="black"), align="center"
    )

# Final layout
dashboard.update_layout(
    height=800, width=1200, title_text="Customer Churn Dashboard", title_x=0.5,
    template="plotly_white", legend=dict(orientation="h", y=-0.1, x=0.3),
    margin=dict(l=50, r=50, t=120, b=50)
)

dashboard.show()

# Save dashboard
output_dir = "../visualizations/interactive"
os.makedirs(output_dir, exist_ok=True)
dashboard.write_html(os.path.join(output_dir, "churn_dashboard.html"))